# Run a test calculation with `a-bam-p`/`acd-1-p` to find gaps in our workflow...

In [4]:
import logging
import openforcefield
print(openforcefield.__version__)

from openforcefield.typing.engines.smirnoff import ForceField, unit

0.0.4


In [5]:
import openeye.oechem
print(openeye.__version__)

from openeye.oechem import (
    oemolistream, oemolostream, OEIFlavor_MOL2_Forcefield,
    OEIFlavor_Generic_Default, OEIFlavor_PDB_Default, OEIFlavor_PDB_ALL,
    OEFormat_MOL2, OEFormat_MOL2H, OEWriteMolecule, OETriposAtomNames, OEMol,
    OEFormat_PDB, OESmilesToMol, OEAddExplicitHydrogens, OEHasAtomIdx,
    OEAtomGetResidue)

2018.Feb.1


In [6]:
import parmed as pmd
print(pmd.__version__)

2.7.3


In [7]:
def load_mol2(filename, name=None, add_tripos=True, flavor='FF'):
    """
    Converts a `mol2` file to an `OEMol` object.
    Parameters
    ----------
    filename : str
        MOL2 file
    name : str
        Residue name
    add_tripos : bool
        Whether to add Tripos atom names to the file
    Returns
    -------
    openeye.oechem.OEMol
    """
    logging.info(f'Loading {filename}...')
    ifs = oemolistream()
    if flavor is not None:
        flavor = OEIFlavor_MOL2_Forcefield
        ifs.SetFlavor(OEFormat_MOL2, flavor)
    molecules = []

    if not ifs.open(filename):
        logging.error(f'Unable to open {filename} for reading...')

    for mol in ifs.GetOEMols():
        if add_tripos:
            OETriposAtomNames(mol)
        if name:
            mol.SetTitle(name)
        # Add all the molecules in this file to a list, but only return the first one.
        molecules.append(OEMol(mol))
        # This should now handle single-residue and multi-residue hosts.
        return molecules[0]

In [14]:
path = './benchmarksets/input_files/cd-set1//prmtop-rst7/smirnoff/acd-1-p/'
host = load_mol2(path + 'MGO-sybyl.mol2', add_tripos=True)
guest = load_mol2(path + 'MOL-sybyl.mol2')

INFO:root:Loading ./benchmarksets/input_files/cd-set1//prmtop-rst7/smirnoff/acd-1-p/MGO-sybyl.mol2...
INFO:root:Loading ./benchmarksets/input_files/cd-set1//prmtop-rst7/smirnoff/acd-1-p/MOL-sybyl.mol2...


In [15]:
molecules = [host, guest]

In [16]:
def create_host_guest_topology(components, host_resname, guest_resname):
    """Return the topology components belonging the host and guest only.
    
    Parameters:
    ----------
    components : parmed.topology.components
        ParmEd topology components, split by molecule
    host_resname : str
        Residue name of the host molecule (no colon)
    guest_resname : str
        Residue name of the guest molecule (no colon)
    Returns
    -------
    pmd.Structure
        A ParmEd structure containing just the host and guest molecule
    """
    logging.info(
        'Creating a combined topology for the host and guest molecules...')
    topology = pmd.Structure()
    for component in components:
        # Check the first residue of each component because there may be multiple residues in each component, but they should all have the same residue name.hash
        if component[0].residues[0].name == host_resname.upper() or \
            component[0].residues[0].name == guest_resname.upper():
            topology += component[0]
    return topology

In [17]:
# Either loading acd-1-p.prmtop and acd-1-p.rst7 or loading smirnoff.pruned.pdb (created from those files) casues an error.
input_topology = pmd.load_file(path + 'smirnoff.pruned.pdb')

In [18]:
topology = create_host_guest_topology(input_topology.split(), 'MGO', 'MOL')

INFO:root:Creating a combined topology for the host and guest molecules...


In [19]:
ff = ForceField('forcefield/smirnoff99Frosst.offxml')
system = ff.createSystem(
    topology,
    molecules,
    nonbondedCutoff=1.1 * unit.nanometer,
    ewaldErrorTolerance=1e-4)

AttributeError: 'Structure' object has no attribute '_chains'

## Load in the reference coordinates and SMIRNOFF99Frosst parameters

In [57]:
def my_convert(destination,
            prefix,
            reference_prmtop,
            reference_inpcrd,
            host_resname,
            guest_resname,
            debug=False):
    """
    Convert from an existing parameter set to SMIRNOFF99Frosst.
    Parameters
    ----------
    destination : str
        Directory where existing files will be read and new files will be written
    prefix : str
        Base name of output files (e.g., "smirnoff" or "hg")
    reference_prmtop : str
        Name of existing AMBER parameter file
    reference_inpcrd : str
        Name of existing AMBER coordinate file
    host_resname : str
        Residue name of the host molecule (*not* the mask)
    guest_resname : str
        Residue name of the guest molecule (*not* the mask)
    debug : bool
        If True, intermediary files will be kept
    Returns
    -------
    merged : parmed.Structure
        A ParmEd structure containing the SMIRNOFF99Frosst parameters and coordinates
    """

    reference = pmd.load_file(
        destination + reference_prmtop, xyz=destination + reference_inpcrd)
    
    components = split_topology(file_name=destination + prefix + '.pruned.pdb')
    hg_topology = create_host_guest_topology(
        components, host_resname=host_resname, guest_resname=guest_resname)

    host = load_mol2(
        filename=destination + host_resname + '-sybyl.mol2',
        name=host_resname,
        add_tripos=True)

    guest = load_mol2(
        filename=destination + guest_resname + '-sybyl.mol2',
        name=guest_resname,
        add_tripos=False)

    molecules = [host, guest]

    ff = ForceField('forcefield/smirnoff99Frosst.offxml')
    system = ff.createSystem(
        hg_topology.topology,
        molecules,
        nonbondedCutoff=1.1 * unit.nanometer,
        ewaldErrorTolerance=1e-4)

In [58]:
nonunique = my_convert(destination, name, 
    reference_prmtop, reference_inpcrd, 
    host_resname, guest_resname,
               debug=True)

INFO:root:Splitting topology into components...
INFO:root:Creating a combined topology for the host and guest molecules...
INFO:root:Loading ./benchmarksets/input_files/cd-set1//prmtop-rst7/smirnoff/acd-1-p/MGO-sybyl.mol2...
INFO:root:Loading ./benchmarksets/input_files/cd-set1//prmtop-rst7/smirnoff/acd-1-p/MOL-sybyl.mol2...


<openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fe03ccb2ae0> >
<openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fe02d240cf0> >
<Structure 143 atoms; 7 residues; 148 bonds; NOT parametrized>


SMIRKSParsingError: (SMIRKSParsingError(...), 'Error SMIRKS ([*:1]-[#6X4:2]-[#6X4:3]-[*:4]) was not parseable                        into a ChemicalEnvironment')

In [ ]:
from smirnovert.amber import create_mapping

In [10]:
host_mapping, guest_mapping = create_mapping(nonunique, host_resname, guest_resname)

In [14]:
from smirnovert.amber import remap_atom_types

In [15]:
parm = pmd.amber.LoadParm(destination + 'smirnoff.prmtop')

In [16]:
remap_atom_types(parm, host_resname, host_mapping, guest_resname, guest_mapping, destination)

Assigning 0 C1 1 → C0
Assigning 0 H1 2 → H0
Assigning 0 O1 3 → O0
Assigning 0 C2 4 → C1
Assigning 0 H2 5 → H1
Assigning 0 O2 6 → O1
Assigning 0 HO2 7 → H2
Assigning 0 C3 8 → C2
Assigning 0 H3 9 → H3
Assigning 0 O3 10 → O2
Assigning 0 HO3 11 → H4
Assigning 0 C4 12 → C3
Assigning 0 H4 13 → H5
Assigning 0 C5 14 → C4
Assigning 0 H5 15 → H6
Assigning 0 O5 16 → O3
Assigning 0 C6 17 → C5
Assigning 0 H61 18 → H7
Assigning 0 H62 19 → H8
Assigning 0 O6 20 → O4
Assigning 0 HO6 21 → H9
Assigning 1 C1 22 → C0
Assigning 1 H1 23 → H0
Assigning 1 O1 24 → O0
Assigning 1 C2 25 → C1
Assigning 1 H2 26 → H1
Assigning 1 O2 27 → O1
Assigning 1 HO2 28 → H2
Assigning 1 C3 29 → C2
Assigning 1 H3 30 → H3
Assigning 1 O3 31 → O2
Assigning 1 HO3 32 → H4
Assigning 1 C4 33 → C3
Assigning 1 H4 34 → H5
Assigning 1 C5 35 → C4
Assigning 1 H5 36 → H6
Assigning 1 O5 37 → O3
Assigning 1 C6 38 → C5
Assigning 1 H61 39 → H7
Assigning 1 H62 40 → H8
Assigning 1 O6 41 → O4
Assigning 1 HO6 42 → H9
Assigning 2 C1 43 → C0
Assigning 

## Now re-load in the new structure

In [21]:
hg = pmd.load_file(destination + 'smirnoff-unique.prmtop', destination + 'smirnoff.inpcrd', structure=True)

## Align

In [22]:
# Find the guest atoms to align by reading first line in /data/nhenriksen/projects/cds/wat6/bgbg-tip3p/pdbs
hg = align.zalign(hg, ':MOL@C4', ':MOL@N1', save=True, filename=destination + 'hg-aligned.pdb')

INFO:root:Moving :MOL@C4 (1 atoms) to the origin...
INFO:root:Aligning :MOL@N1 (1 atoms) with the z axis...
INFO:root:Saved aligned coordinates to ./benchmarksets/input_files/cd-set1//prmtop-rst7/smirnoff/acd-1-p/hg-aligned.pdb


## Add dummy atoms

In [23]:
hg = pmd.load_file(destination + 'hg-aligned.pdb', structure=True)
# print(hg.atoms[-1].number)

hg = dummy.add_dummy(hg, residue_name='DM1', z=-6.000)
hg = dummy.add_dummy(hg, residue_name='DM2', z=-9.000)
hg = dummy.add_dummy(hg, residue_name='DM3', z=-11.200, y=2.200)

dummy.write_dummy_frcmod(path = destination)
dummy.write_dummy_mol2(path = destination, residue_name='DM1', filename='DM1.mol2')
dummy.write_dummy_mol2(path = destination, residue_name='DM2', filename='DM2.mol2')
dummy.write_dummy_mol2(path = destination, residue_name='DM3', filename='DM3.mol2')

hg.write_pdb(destination + 'hg-aligned-dum.pdb', renumber=False)

## Solvate

In [24]:
with open(destination + 'tleap.in', 'w') as file:
    tleap_header = f'''\n
source leaprc.water.tip3p
loadamberparams {base_name}-unique.frcmod
{host_resname} = loadmol2 {base_name}-{host_resname}-multi-unique.mol2
{guest_resname} = loadmol2 {base_name}-{guest_resname}-unique.mol2

DM1 = loadmol2 DM1.mol2
DM2 = loadmol2 DM2.mol2
DM3 = loadmol2 DM3.mol2
loadamberparams dummy.frcmod

model = loadpdb {str('hg-aligned-dum.pdb')}
bond model.1.O1 model.2.C4
bond model.2.O1 model.3.C4
bond model.3.O1 model.4.C4
bond model.4.O1 model.5.C4
bond model.5.O1 model.6.C4
bond model.6.O1 model.1.C4


'''
    file.write(tleap_header)
#     for line in bonds_between_monomers:
#         file.write(line + '\n')

In [32]:
build.solvate(tleap_file='tleap.in', pdb_file='hg-aligned-dum.pdb', pbc_type='rectangular', buffer_target=2000,
             output_prefix='hg-solvated', path=destination)

DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 5782, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 00	 1 5782 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 01	 -4 1499 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 02	 -3 1499 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 03	 -3 1499 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 04	 -3 1499 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 05	 -3 1499 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1, 'DM3': 1}
DEBUG:root:Cycle 06	 -3 1499 (2000)
DEBUG:root:{'MGO': 6, 'MOL': 1, 'Cl-': 4, 'Na+': 4, 'WAT': 1499, 'DM1': 1, 'DM2': 1,

Exception: Automatic adjustment of the buffer value resulted in fewer waters             added than targeted by `buffer_water`. Try increasing the tolerance in the above loop